In [36]:
from google.cloud import aiplatform
from google.cloud import bigquery

import pandas as pd
import numpy as np
from datetime import date,datetime,timedelta

import math
import os

import tensorflow as tf
import tensorflow_decision_forests as tfdf  # constantly registered to load model 
print(tf.__version__)
print(tfdf.__version__)

2.12.0
1.4.0


In [37]:
# import functions_framework
# @functions_framework.http
# def xgb_tf_predict_incident_severity(request):

In [38]:
model_tree_type=1 # 1= xgboost  2-random forest
option=1

if model_tree_type==1:
    _model='model_xgb_tf'
else:
     _model='model_rf_tf'

model_gs_path=f"gs://demo-tuned-tf-incident-pongthorn/{_model}"
print(model_gs_path)

projectId="pongthorn"
dataset_id="SMartML"

map_severity_to_class={0:4,1: 3, 2: 2, 3: 1}

if option==1:
    unusedCols_unseen=['id','severity_name','open_to_close_hour']
else:
    unusedCols_unseen=['id','severity_name','range_open_to_close_hour']
    

gs://demo-tuned-tf-incident-pongthorn/model_xgb_tf


# Load Model

In [39]:
abc_model = tf.keras.models.load_model(model_gs_path)  
print(abc_model.summary())
# abc_model = tf.keras.models.load_model(model_local_path) 

[INFO 23-07-07 15:42:36.6887 UTC kernel.cc:1243] Loading model from path gs://demo-tuned-tf-incident-pongthorn/model_xgb_tf/assets/ with prefix 3ac63f3679524a2a
[INFO 23-07-07 15:42:37.1656 UTC decision_forest.cc:660] Model loaded with 148 root(s), 7318 node(s), and 6 input feature(s).
[INFO 23-07-07 15:42:37.1665 UTC abstract_model.cc:1311] Engine "GradientBoostedTreesGeneric" built
[INFO 23-07-07 15:42:37.1670 UTC kernel.cc:1075] Use fast generic engine


Model: "gradient_boosted_trees_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1
Trainable params: 0
Non-trainable params: 1
_________________________________________________________________
None


# Make Prediction

In [40]:
client = bigquery.Client(project=projectId)
new_data_table_id=f"{projectId}.{dataset_id}.new2_incident"

query_result=client.query(f"SELECT * FROM {new_data_table_id}")
df=query_result.to_dataframe()

unseen =df.drop(columns=unusedCols_unseen)
print(unseen.info())
unseen.tail(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   severity_id               17 non-null     Int64 
 1   sla                       17 non-null     object
 2   product_type              17 non-null     object
 3   brand                     17 non-null     object
 4   service_type              17 non-null     object
 5   incident_type             17 non-null     object
 6   range_open_to_close_hour  17 non-null     object
dtypes: Int64(1), object(6)
memory usage: 1.1+ KB
None


,severity_id,sla,product_type,brand,service_type,incident_type,range_open_to_close_hour
7,3,24x7 4Hrs Response Time,Storage,NetApp,Incident,Controller/Node Failure,soon
8,3,24x7 4Hrs Resolution Time,Storage,NetApp,Incident,Controller/Node Failure,soon
9,3,24x7 4Hrs Resolution Time,Storage,NetApp,Incident,Hard Disk Drive Failure,latest
10,3,24x7 4Hrs Response Time,Storage,NetApp,Incident,Hard Disk Drive Failure,soon
11,4,24x7 4Hrs Response Time,Storage,NetApp,Incident,General Incident,soon
12,3,24x7 4Hrs Response Time,Security,Trend Micro,Incident,Upgrade Software,soon
13,3,24x7 4Hrs Response Time,Software,VMWare,Incident,General Incident,soonest
14,3,24x7 4Hrs Response Time,Software,VMWare,Incident,General Incident,soonest
15,3,24x7 6Hrs Resolution Time,Software,eUnite,Incident,Software,soonest
16,2,24x7 6Hrs Resolution Time,Software,eUnite,Incident,Software,soonest


In [42]:
unseen_ds= tfdf.keras.pd_dataframe_to_tf_dataset(unseen.drop(columns=['severity_id']))
print(unseen_ds)

<_PrefetchDataset element_spec={'sla': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'product_type': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'brand': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'service_type': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'incident_type': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'range_open_to_close_hour': TensorSpec(shape=(None,), dtype=tf.string, name=None)}>


In [43]:
predResultList=abc_model.predict(unseen_ds)
predServerityIDList=[]
for predResult in predResultList:
    _class=tf.argmax(predResult,-1).numpy()
    pred_seveirty_id=map_severity_to_class[_class]
    predServerityIDList.append(pred_seveirty_id)
    print(f"{predResult} : {_class} as severity#{pred_seveirty_id}")
    
dfPred=pd.DataFrame(data=predServerityIDList,columns=["pred_severity_id"])   
df=pd.concat([dfPred,df],axis=1)
# df

1/1 [==============================] - 0s 74ms/step
[0.94347364 0.0481788  0.00486785 0.00347972] : 0 as severity#4
[0.01779265 0.08065245 0.8698259  0.03172899] : 2 as severity#2
[0.02062506 0.12977083 0.8195875  0.03001656] : 2 as severity#2
[0.00982261 0.04917946 0.9345066  0.0064913 ] : 2 as severity#2
[0.02177456 0.06510144 0.8986616  0.01446236] : 2 as severity#2
[0.01796301 0.07064571 0.8959037  0.0154876 ] : 2 as severity#2
[0.0202044  0.33375847 0.6294168  0.0166203 ] : 2 as severity#2
[0.0084537  0.5115585  0.3211109  0.15887694] : 1 as severity#3
[0.01782154 0.24195981 0.26666096 0.47355765] : 3 as severity#1
[0.01380757 0.7401283  0.21320215 0.03286188] : 1 as severity#3
[0.01318795 0.4098764  0.5532594  0.02367628] : 2 as severity#2
[0.01558995 0.7628804  0.208927   0.01260272] : 1 as severity#3
[0.01177808 0.9077274  0.07633034 0.00416415] : 1 as severity#3
[0.0132471  0.5037367  0.46594632 0.01706991] : 1 as severity#3
[0.0132471  0.5037367  0.46594632 0.01706991] : 1 as

2023-07-07 15:43:00.623529: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [17]
	 [[{{node Placeholder/_0}}]]


# Evaluate model

In [44]:
# from sklearn.metrics import confusion_matrix,classification_report
# className=list(set().union(list(df['pred_severity_id'].unique()),list(df['severity_id'].unique())))
# className
# actualClass=[  f'actual-{x}' for x in  className]
# predictedlClass=[  f'pred-{x}' for x in className]
# y_true=list(df['severity_id'])
# y_pred=list(df['pred_severity_id'])
# cnf_matrix = confusion_matrix(y_true,y_pred)
# cnf_matrix

# # #index=actual , column=prediction
# cm_df = pd.DataFrame(cnf_matrix,
#                      index = actualClass, 
#                      columns = predictedlClass)
# print(cm_df)
# print(classification_report(y_true, y_pred, labels=className))

# Write Prediction Result to BQ

In [45]:
df=df[['id','severity_id','pred_severity_id']]
df['prediction_datetime']=datetime.now()
print(df)

      id  severity_id  pred_severity_id        prediction_datetime
0   2790            4                 4 2023-07-07 15:43:07.472917
1   3027            2                 2 2023-07-07 15:43:07.472917
2   2832            2                 2 2023-07-07 15:43:07.472917
3   3026            2                 2 2023-07-07 15:43:07.472917
4   3025            2                 2 2023-07-07 15:43:07.472917
5   3021            3                 2 2023-07-07 15:43:07.472917
6   3029            4                 2 2023-07-07 15:43:07.472917
7   3034            3                 3 2023-07-07 15:43:07.472917
8   3035            3                 1 2023-07-07 15:43:07.472917
9   3032            3                 3 2023-07-07 15:43:07.472917
10  3030            3                 2 2023-07-07 15:43:07.472917
11  2957            4                 3 2023-07-07 15:43:07.472917
12  3031            3                 3 2023-07-07 15:43:07.472917
13  3033            3                 3 2023-07-07 15:43:07.47

In [46]:
predictResult_table_id=f"{projectId}.{dataset_id}.new2_result_prediction_incident"

try:
    client.get_table(predictResult_table_id)  # Make an API request.
    print("Predict Result Table {} already exists.".format(predictResult_table_id))
except Exception as ex:
    schema = [
    bigquery.SchemaField("id", "INTEGER", mode="REQUIRED"),
    bigquery.SchemaField("severity_id", "INTEGER", mode="REQUIRED"),
    bigquery.SchemaField("pred_severity_id", "INTEGER", mode="REQUIRED"),       
    bigquery.SchemaField("prediction_datetime", "DATETIME", mode="REQUIRED") 
    ]

    table = bigquery.Table(predictResult_table_id,schema=schema)
    # table.time_partitioning = bigquery.TimePartitioning(
    # type_=bigquery.TimePartitioningType.DAY,field="prediction_item_date")
    
    table = client.create_table(table)  # Make an API request.
    
    print(
        "Created table {}.{}.{}".format(table.project, table.dataset_id, table.table_id)
    )

Predict Result Table pongthorn.SMartML.new2_result_prediction_incident already exists.


In [47]:
def loadDataFrameToBQ():
    try:
        job_config = bigquery.LoadJobConfig(
            write_disposition="WRITE_APPEND",
        )

        job = client.load_table_from_dataframe(
            df, predictResult_table_id, job_config=job_config
        )
        job.result()  # Wait for the job to complete.
        print("Total Prediction ML ", len(df), "Imported bigquery successfully")

    except BadRequest as e:
        print("Bigquery Error\n")
        for e in job.errors:
            print('ERROR: {}'.format(e['message']))

try:
    loadDataFrameToBQ()
except Exception as ex:
    raise ex

Total Prediction ML  17 Imported bigquery successfully


In [ ]:
# return 'All incidents has been predicted completely.'